In [11]:
import sys, os

import torch
import torch.nn as nn
from torch.utils.data import (
    Dataset, RandomSampler, DataLoader)

from dataset.mnist import load_mnist


sys.path.append(os.pardir)

(x_train, y_train), (x_test, y_test) = load_mnist(flatten=True, normalize=True, one_hot_label=False)

x_train, y_train, x_test, y_test = (
    torch.Tensor(d) for d in (x_train, y_train, x_test, y_test)
)

y_train = y_train.long()
y_test = y_test.long()

In [12]:
class MnistDataset(Dataset):
    
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __len__(self):
        return self.x.size(0)
    
    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

In [13]:
train_dataset = MnistDataset(x_train, y_train)
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)

test_dataset = MnistDataset(x_test, y_test)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [14]:
device = 'cpu'
# device = 'cuda'

In [15]:
model = nn.Sequential(
    nn.Linear(784, 100, bias=True), nn.ReLU(),
    nn.Linear(100, 50, bias=True), nn.ReLU(),
    nn.Linear(50, 10, bias=True), nn.Softmax(dim=-1)
).to(device)

In [16]:
optimizer = torch.optim.SGD(model.parameters(), lr=1e-2, momentum=0.5)

In [17]:
loss_fn = nn.CrossEntropyLoss()

In [18]:
EPOCHS = 60
n = 60000
bsz = train_dataloader.batch_size
for i in range(EPOCHS):
    correct = 0
    loss = 0
    for batch_idx, sample in enumerate(train_dataloader):
        x, y = sample[0].to(device), sample[1].to(device)
        optimizer.zero_grad()
        y_pred = model(x)
        loss = loss_fn(y_pred, y)
        loss.backward()
        optimizer.step()
        correct += torch.eq(torch.argmax(y_pred, dim=-1), y).sum().item()
        loss += loss.item()
    print(f'\rEPOCH: [{i+1:03}/{EPOCHS:03}] Acc={correct/n:.2%} Loss={loss/bsz:.4f}')

EPOCH: [001/060] Acc=16.17% Loss=0.0717
EPOCH: [002/060] Acc=32.87% Loss=0.0712
EPOCH: [003/060] Acc=45.41% Loss=0.0616
EPOCH: [004/060] Acc=68.18% Loss=0.0576
EPOCH: [005/060] Acc=73.26% Loss=0.0518
EPOCH: [006/060] Acc=74.36% Loss=0.0527
EPOCH: [007/060] Acc=74.90% Loss=0.0521
EPOCH: [008/060] Acc=75.22% Loss=0.0564
EPOCH: [009/060] Acc=75.53% Loss=0.0593
EPOCH: [010/060] Acc=75.72% Loss=0.0509
EPOCH: [011/060] Acc=75.87% Loss=0.0536
EPOCH: [012/060] Acc=75.97% Loss=0.0514
EPOCH: [013/060] Acc=76.14% Loss=0.0520
EPOCH: [014/060] Acc=78.35% Loss=0.0506
EPOCH: [015/060] Acc=82.68% Loss=0.0487
EPOCH: [016/060] Acc=83.71% Loss=0.0520
EPOCH: [017/060] Acc=84.26% Loss=0.0491
EPOCH: [018/060] Acc=84.58% Loss=0.0492
EPOCH: [019/060] Acc=84.83% Loss=0.0491
EPOCH: [020/060] Acc=85.02% Loss=0.0506
EPOCH: [021/060] Acc=85.19% Loss=0.0502
EPOCH: [022/060] Acc=85.32% Loss=0.0495
EPOCH: [023/060] Acc=85.46% Loss=0.0495
EPOCH: [024/060] Acc=85.59% Loss=0.0473
EPOCH: [025/060] Acc=85.68% Loss=0.0497


In [19]:
model.eval()

Sequential(
  (0): Linear(in_features=784, out_features=100, bias=True)
  (1): ReLU()
  (2): Linear(in_features=100, out_features=50, bias=True)
  (3): ReLU()
  (4): Linear(in_features=50, out_features=10, bias=True)
  (5): Softmax(dim=-1)
)

In [34]:
correct = 0
for sample in test_dataloader:
    x, y = sample[0].to(device), sample[1].to(device)
    y_pred = model(x)
    correct += torch.eq(torch.argmax(y_pred, dim=-1), y).sum()

In [35]:
correct.sum().item() / y_test.size(0)

0.8719